# This notebook explores valley bottom extraction methods

existing:

    - V-BET:
    - Geomorphons (WhiteBoxTools implementation)
    - CostAccumulation:
    - Slope Threshold:
    - Height Threshold:
    - FluvialCorridor
    - Valley Confinement Algorithm

totest:

- SurfaceVoronoi
- CostAccumulation delineation (danish paper, depth to water, downslope distance to stream, elevation above stream, wetness index, hillslope extraction)
- Adaptive threshold of slope and height
- curvature and wavelet analysis (smooth first with gaussian), cross corrolate wavelet with curvature (oskin)
- nonlinear diffusion hillsope models to get sense of different cross sections for river valleys (oskin)
- topographic openness (oskin)
- get median aspect of valley to help delineate slope and hillslope?
  

In [68]:
from dataclasses import dataclass
import glob
import os
import shutil

import leafmap.leafmap as leafmap
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import py3dep 
from pynhd import NHD
import rasterio as rio
from rasterio import Affine
from rasterio.plot import show
import whitebox

In [12]:
@dataclass
class Raster:
    data: np.ndarray
    metadata: dict
    
    @classmethod
    def from_file(cls, raster_file):
        with rio.open(raster_file) as dataset:
            data = dataset.read(1)
            metadata = dataset.meta.copy()
                
            if "float" in str(data.dtype):
                metadata['nodata'] = np.nan if metadata['nodata'] is None else metadata['nodata']
                
                data[data > 3.0e38] = metadata['nodata']
                data[data < -3.0e38] = metadata['nodata']
                
            return cls(data, metadata)
        
    @classmethod
    def from_xarray_rio(cls, xarr):
        metadata = {
            'count': xarr.rio.count,
            'height': xarr.rio.height,
            'width': xarr.rio.width,
            'dtype': str(xarr.dtype),
            'crs': xarr.rio.crs,
            'nodata': xarr.rio.nodata,
            'transform': xarr.rio.transform()
        }
        data = xarr.values
        return cls(data, metadata)
        
    def to_file(self, output_file):
        with rio.open(output_file, 'w', **self.metadata) as dataset:
            dataset.write(self.data,1)
                
    def zero_to_nodata(self):
        self.data[self.data == 0] = self.metadata['nodata']
        self.data[self.data == 0] = self.metadata['nodata']
    
    def plot(self):
        show(self.data)        

In [52]:
# paths and configuration
valley_polygons_path = "../raw_data/sample_valley_polygons/sample_valleys.geojson"

working_directory = "../data/valley_bottom_testing/"

## Get DEM and Flowline Data (run once)

In [21]:
if os.path.isdir(working_directory):
    shutil.rmtree(working_directory)
os.mkdir(working_directory)

In [22]:
gdf = gpd.read_file(valley_polygons_path, driver='GeoJSON')
gdf = gdf[["layer", "geometry"]]
gdf = gdf.explode('geometry', index_parts=False)
gdf

,layer,geometry
0,russian_confined,"POLYGON ((6257387.991 1939912.155, 6254095.915..."
1,russian_large,"POLYGON ((6231409.111 1908836.263, 6231409.111..."
2,russian_med,"POLYGON ((6270261.608 1935260.309, 6270261.608..."
3,russian_partly_confined,"POLYGON ((6246509.827 1923216.026, 6246829.773..."
4,russian_small,"POLYGON ((6292388.399 1925695.607, 6292388.399..."
5,russian_unconfined,"POLYGON ((6273705.237 1945469.112, 6272021.311..."
6,santa_ynez_confined,"POLYGON ((7070336.506 552287.319, 7070800.663 ..."
7,santa_ynez_large,"POLYGON ((7071756.826 517740.130, 7071756.826 ..."
8,santa_ynez_med,"POLYGON ((7025749.607 485509.084, 7025749.607 ..."
9,santa_ynez_partly_confined,"POLYGON ((7082898.909 545930.692, 7083446.614 ..."


In [33]:
# get dems and save to files
def get_dem(geometry):
    dem = py3dep.get_map("DEM", geometry, resolution=10, geo_crs="epsg:6418", crs="epsg:4326")
    dem.rio.reproject(6418)
    dem = Raster.from_xarray_rio(dem)
    return dem

dem_filepaths = []
for index,row in gdf.iterrows():
    print(f"getting dem for {row['layer']}")
    dem = get_dem(row['geometry'])
    fname = os.path.join(working_directory, f"{row['layer']}-dem30m.tif")
    dem_filepaths.append(fname)
    dem.to_file(fname)

getting dem for russian_confined
getting dem for russian_large
getting dem for russian_med
getting dem for russian_partly_confined
getting dem for russian_small
getting dem for russian_unconfined
getting dem for santa_ynez_confined
getting dem for santa_ynez_large
getting dem for santa_ynez_med
getting dem for santa_ynez_partly_confined
getting dem for santa_ynez_small
getting dem for santa_ynez_unconfined
getting dem for sierra_confined
getting dem for sierra_large
getting dem for sierra_med
getting dem for sierra_partly_confined
getting dem for sierra_small
getting dem for sierra_unconfined


In [34]:
# get NHD flowlines
def get_flowlines(geometry):
    fields = [
        'COMID',
    ]

    nhd = NHD('flowline_mr', outfields = fields)
    try:
        flowlines = nhd.bygeom(geometry, geo_crs=6418)
        flowlines = flowlines.to_crs(6418)
    except Exception as e:
        flowlines = None
    return flowlines
    
flowline_filepaths = []
for index,row in gdf.iterrows():
    print(f"getting flowlines for {row['layer']}")
    flowlines = get_flowlines(row['geometry'])
    fname = os.path.join(working_directory, f"{row['layer']}-flowlines.geojson")
    if flowlines is not None:
        flowlines.to_file(fname, driver="GeoJSON")
        flowline_filepaths.append(fname)
    else:
        print(f"\tno flowlines for {row['layer']}")
        flowline_filepaths.append(None)

getting flowlines for russian_confined
getting flowlines for russian_large
getting flowlines for russian_med
getting flowlines for russian_partly_confined
getting flowlines for russian_small
getting flowlines for russian_unconfined
getting flowlines for santa_ynez_confined
	no flowlines for santa_ynez_confined
getting flowlines for santa_ynez_large
getting flowlines for santa_ynez_med
getting flowlines for santa_ynez_partly_confined
getting flowlines for santa_ynez_small
getting flowlines for santa_ynez_unconfined
getting flowlines for sierra_confined
getting flowlines for sierra_large
getting flowlines for sierra_med
getting flowlines for sierra_partly_confined
getting flowlines for sierra_small
getting flowlines for sierra_unconfined


In [41]:
gdf["dem_filepath"] = dem_filepaths
gdf["flowline_filepath"] = flowline_filepaths
gdf.to_file(os.path.join(working_directory, 'manifest.geojson'), driver='GeoJSON')  

In [46]:
# all leafmap stuff seems to be in EPSG:4326
# plot Russian med

dem_fpath = gdf.iloc[2]["dem_filepath"]
flow_fpath = gdf.iloc[2]["flowline_filepath"]

m = leafmap.Map(center=[38.5, -122.95], zoom=12)
m.add_gdf(gpd.read_file(flow_fpath), layer_name='Flowlines')
m.add_raster(dem_fpath, cmap="terrain", layer_name="DEM")

m

Map(center=[38.5, -122.95], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

# Load Data

In [47]:
manifest = gpd.read_file(os.path.join(working_directory, "manifest.geojson"))
manifest

,layer,dem_filepath,flowline_filepath,geometry
0,russian_confined,../data/valley_bottom_testing/russian_confined...,../data/valley_bottom_testing/russian_confined...,"POLYGON ((6257387.991 1939912.155, 6254095.915..."
1,russian_large,../data/valley_bottom_testing/russian_large-de...,../data/valley_bottom_testing/russian_large-fl...,"POLYGON ((6231409.111 1908836.263, 6231409.111..."
2,russian_med,../data/valley_bottom_testing/russian_med-dem3...,../data/valley_bottom_testing/russian_med-flow...,"POLYGON ((6270261.608 1935260.309, 6270261.608..."
3,russian_partly_confined,../data/valley_bottom_testing/russian_partly_c...,../data/valley_bottom_testing/russian_partly_c...,"POLYGON ((6246509.827 1923216.026, 6246829.773..."
4,russian_small,../data/valley_bottom_testing/russian_small-de...,../data/valley_bottom_testing/russian_small-fl...,"POLYGON ((6292388.399 1925695.607, 6292388.399..."
5,russian_unconfined,../data/valley_bottom_testing/russian_unconfin...,../data/valley_bottom_testing/russian_unconfin...,"POLYGON ((6273705.237 1945469.112, 6272021.311..."
6,santa_ynez_confined,../data/valley_bottom_testing/santa_ynez_confi...,NaN,"POLYGON ((7070336.506 552287.319, 7070800.663 ..."
7,santa_ynez_large,../data/valley_bottom_testing/santa_ynez_large...,../data/valley_bottom_testing/santa_ynez_large...,"POLYGON ((7071756.826 517740.130, 7071756.826 ..."
8,santa_ynez_med,../data/valley_bottom_testing/santa_ynez_med-d...,../data/valley_bottom_testing/santa_ynez_med-f...,"POLYGON ((7025749.607 485509.084, 7025749.607 ..."
9,santa_ynez_partly_confined,../data/valley_bottom_testing/santa_ynez_partl...,../data/valley_bottom_testing/santa_ynez_partl...,"POLYGON ((7082898.909 545930.692, 7083446.614 ..."


# Geomorphons

paper: https://www.sciencedirect.com/science/article/pii/S0169555X12005028

whiteboxtools implementation: https://www.whiteboxgeo.com/manual/wbt_book/available_tools/geomorphometric_analysis.html#geomorphons

classes picture:


In [65]:
wbt = whitebox.WhiteboxTools()
wbt.set_whitebox_dir("/Users/arthurkoehl/opt/WBT/")

wbt.set_working_dir(os.path.abspath(working_directory))
wbt.version()
def my_callback(value):
    if "Elapsed Time" in value:
        print(value)

In [62]:
# geomorphons
geomorphon_filepaths = []
for index,row in manifest.iterrows():
    wbt.geomorphons(
        os.path.basename(row['dem_filepath']), 
        f"{row['layer']}-geomorphons.tif", 
        search=50, 
        threshold=0.0, 
        fdist=0, 
        skip=0, 
        forms=True, 
        residuals=False,
        callback=my_callback
    )
    geomorphon_filepaths.append(os.path.join(working_directory, f"{row['layer']}-geomorphons.tif"))

manifest['geomorphons'] = geomorphon_filepaths
manifest

./whitebox_tools --run="Geomorphons" --wd="/Users/arthurkoehl/programs/pasternack/valleys/data/valley_bottom_testing" --dem='russian_confined-dem30m.tif' --output='russian_confined-geomorphons.tif' --search=50 --threshold=0.0 --fdist=0 --skip=0 --forms -v --compress_rasters=False

****************************
* Welcome to Geomorphons   *
* Powered by WhiteboxTools *
* www.whiteboxgeo.com      *
****************************
Reading data...
Generating global ternary codes...
Computing geomorphons...
Saving data...
Output file written
Elapsed Time (excluding I/O): 1.154s
./whitebox_tools --run="Geomorphons" --wd="/Users/arthurkoehl/programs/pasternack/valleys/data/valley_bottom_testing" --dem='russian_large-dem30m.tif' --output='russian_large-geomorphons.tif' --search=50 --threshold=0.0 --fdist=0 --skip=0 --forms -v --compress_rasters=False

****************************
* Welcome to Geomorphons   *
* Powered by WhiteboxTools *
* www.whiteboxgeo.com      *
****************************
Read

,layer,dem_filepath,flowline_filepath,geometry,geomorphons
0,russian_confined,../data/valley_bottom_testing/russian_confined...,../data/valley_bottom_testing/russian_confined...,"POLYGON ((6257387.991 1939912.155, 6254095.915...",../data/valley_bottom_testing/russian_confined...
1,russian_large,../data/valley_bottom_testing/russian_large-de...,../data/valley_bottom_testing/russian_large-fl...,"POLYGON ((6231409.111 1908836.263, 6231409.111...",../data/valley_bottom_testing/russian_large-ge...
2,russian_med,../data/valley_bottom_testing/russian_med-dem3...,../data/valley_bottom_testing/russian_med-flow...,"POLYGON ((6270261.608 1935260.309, 6270261.608...",../data/valley_bottom_testing/russian_med-geom...
3,russian_partly_confined,../data/valley_bottom_testing/russian_partly_c...,../data/valley_bottom_testing/russian_partly_c...,"POLYGON ((6246509.827 1923216.026, 6246829.773...",../data/valley_bottom_testing/russian_partly_c...
4,russian_small,../data/valley_bottom_testing/russian_small-de...,../data/valley_bottom_testing/russian_small-fl...,"POLYGON ((6292388.399 1925695.607, 6292388.399...",../data/valley_bottom_testing/russian_small-ge...
5,russian_unconfined,../data/valley_bottom_testing/russian_unconfin...,../data/valley_bottom_testing/russian_unconfin...,"POLYGON ((6273705.237 1945469.112, 6272021.311...",../data/valley_bottom_testing/russian_unconfin...
6,santa_ynez_confined,../data/valley_bottom_testing/santa_ynez_confi...,NaN,"POLYGON ((7070336.506 552287.319, 7070800.663 ...",../data/valley_bottom_testing/santa_ynez_confi...
7,santa_ynez_large,../data/valley_bottom_testing/santa_ynez_large...,../data/valley_bottom_testing/santa_ynez_large...,"POLYGON ((7071756.826 517740.130, 7071756.826 ...",../data/valley_bottom_testing/santa_ynez_large...
8,santa_ynez_med,../data/valley_bottom_testing/santa_ynez_med-d...,../data/valley_bottom_testing/santa_ynez_med-f...,"POLYGON ((7025749.607 485509.084, 7025749.607 ...",../data/valley_bottom_testing/santa_ynez_med-g...
9,santa_ynez_partly_confined,../data/valley_bottom_testing/santa_ynez_partl...,../data/valley_bottom_testing/santa_ynez_partl...,"POLYGON ((7082898.909 545930.692, 7083446.614 ...",../data/valley_bottom_testing/santa_ynez_partl...


- try larger search distance parameter
- try running on relative elevation model
- test with and without smoothing of the DEM (filter or https://www.whiteboxgeo.com/manual/wbt_book/available_tools/geomorphometric_analysis.html?highlight=feature%20preserving#featurepreservingsmoothing and then of the geomorphon output (median filter)
- combine geomorphon with slope with cost to channel accumulation delineation

clustering based on features for each pixel: (geodesic to channel, slope, curviture, geomorphon, hand, dtw?, relative topographic position)

In [64]:
geomorphons_classes = {
    1: 'Flat',
    2: 'Peak (summit)',
    3:	"Ridge",
    4:	'Shoulder',
    5:	'Spur (convex)',
    6:	'Slope',
    7:	'Hollow (concave)',
    8:	'Footslope',
    9:	'Valley',
    10:	'Pit (depression)',
}

# DEM Preprocessing

Filters:
- Mean Filter
- Median Filter
- Gaussian
- Feature Preserving Filter (whiteboxtools)

In [71]:
manifest.tail()

,layer,dem_filepath,flowline_filepath,geometry,geomorphons
13,sierra_large,../data/valley_bottom_testing/sierra_large-dem...,../data/valley_bottom_testing/sierra_large-flo...,"POLYGON ((7277570.901 1686215.351, 7277570.901...",../data/valley_bottom_testing/sierra_large-geo...
14,sierra_med,../data/valley_bottom_testing/sierra_med-dem30...,../data/valley_bottom_testing/sierra_med-flowl...,"POLYGON ((7035329.317 1899296.905, 7035329.317...",../data/valley_bottom_testing/sierra_med-geomo...
15,sierra_partly_confined,../data/valley_bottom_testing/sierra_partly_co...,../data/valley_bottom_testing/sierra_partly_co...,"POLYGON ((7102084.366 1950143.060, 7102341.165...",../data/valley_bottom_testing/sierra_partly_co...
16,sierra_small,../data/valley_bottom_testing/sierra_small-dem...,../data/valley_bottom_testing/sierra_small-flo...,"POLYGON ((7176678.091 1640039.611, 7176678.091...",../data/valley_bottom_testing/sierra_small-geo...
17,sierra_unconfined,../data/valley_bottom_testing/sierra_unconfine...,../data/valley_bottom_testing/sierra_unconfine...,"POLYGON ((7281081.522 1676088.264, 7285383.954...",../data/valley_bottom_testing/sierra_unconfine...


In [73]:
# use scipy to guassian filter
test_raster = Raster.from_file(manifest[manifest["layer"] == "sierra_med"]["dem_filepath"])

sigma = 0.25

filtered = scipy.ndimage.gaussian_filter(test_raster.data, sigma)

